In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 64kB/s 
     |████████████████████████████████| 204kB 46.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=bd2073d260d109f74e6f7cc650e9b647299112340ee604728ed37de8c74580a4
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
import time
# spark = SparkSession.builder.master("local").getOrCreate()
conf = SparkConf().setAppName('appname').setMaster("local[*]")
conf.set('spark.driver.memory', '40g')
sc = SparkContext.getOrCreate(conf=conf)

In [ ]:
import pandas as pd
taxifile = []
for i in range(9, 10):
  month = ""
  if i < 10:
    month = "0" + str(i)
  else:
    month = str(i)
  url = '/content/drive/Shared drives/NCTU-1091-Big Data/Homework 2/2008.csv'
  taxicsv = pd.read_csv(url, 
              usecols=["Month", "ArrDelay", "DepDelay"],
              dtype={"Month" : "int", "ArrDelay" : "float", "DepDelay" : "float"}
              )
  taxifile.append(taxicsv)
taxifile = pd.concat(taxifile, axis=0, ignore_index=True)
print(taxifile)

         Month  ArrDelay  DepDelay
0            1     -14.0       8.0
1            1       2.0      19.0
2            1      14.0       8.0
3            1      -6.0      -4.0
4            1      34.0      34.0
...        ...       ...       ...
7009723     12      14.0       3.0
7009724     12      -2.0      -1.0
7009725     12       0.0      -5.0
7009726     12       9.0      11.0
7009727     12      -5.0       7.0

[7009728 rows x 3 columns]


In [ ]:
import numpy as np

print(taxifile.loc[178])
print(type(taxifile.loc[178][1]))
if np.isnan(taxifile.loc[178][1]):
  print(True)
else:
  print(False)

Month       1.0
ArrDelay    NaN
DepDelay    NaN
Name: 178, dtype: float64
<class 'numpy.float64'>
True


In [ ]:
from operator import add
def dataProcessing(row):
  if np.isnan(row[1]) and np.isnan(row[2]):
    return [(row[0], -1000), (row[0], -1000)]
  elif np.isnan(row[1]):
    return [(row[0], -1000), (row[0], row[2])]
  elif np.isnan(row[2]):
    return [(row[0], row[1]), (row[0], -1000)]
  else:
    return [(row[0], row[1]), (row[0], row[2])]

location = ["Month", "ArrDelay"]
rddArr = sc.parallelize(taxifile.loc[:, location].values.tolist())
countsArr = (rddArr.reduceByKey(max)
        .collect()
      )
location = ["Month", "DepDelay"]
rddDep = sc.parallelize(taxifile.loc[:, location].values.tolist())
countsDep = (rddDep.reduceByKey(max)
        .collect()
      )

In [ ]:
countsArr = sorted(countsArr, key=lambda x: x[0])
countsDep = sorted(countsDep, key=lambda x: x[0])
print("Month\tArrDelay\tDepDelay")
for i in range(12):
  print("{}\t{}\t\t{}".format(countsArr[i][0], countsArr[i][1], countsDep[i][1]))

# months = [[0, 0] for i in range(13)]
# print(counts[2][1], counts[2][0][0], counts[2][0][1], months[int(counts[2][0][0])][1])
# for i in range(len(counts)):
#   if counts[i][1] > months[int(counts[i][0][0])][1] and counts[i][0][1] != -1000:
#     months[int(counts[i][0][0])][0] = counts[i][0][1]
#     months[int(counts[i][0][0])][1] = counts[i][1]
# for i in range(len(months)):
#   print(months[i])

Month	ArrDelay	DepDelay
1.0	1525.0		1355.0
2.0	2461.0		2457.0
3.0	1490.0		1521.0
4.0	2453.0		2467.0
5.0	1951.0		1952.0
6.0	1707.0		1710.0
7.0	1510.0		1518.0
8.0	1359.0		1367.0
9.0	1583.0		1552.0
10.0	1392.0		1369.0
11.0	1308.0		1286.0
12.0	1655.0		1597.0


In [ ]:
import pandas as pd
flightfile = []
for i in range(0, 6):
  url = '/content/drive/Shared drives/NCTU-1091-Big Data/Homework 2/200' + str(i) + '.csv'
  flightcsv = pd.read_csv(url, 
              usecols=["Year", "WeatherDelay"],
              dtype={"Year" : "int", "WeatherDelay" : "float"}
              )
  flightfile.append(flightcsv)
flightfile = pd.concat(flightfile, axis=0, ignore_index=True)
print(flightfile)

          Year  WeatherDelay
0         2000           NaN
1         2000           NaN
2         2000           NaN
3         2000           NaN
4         2000           NaN
...        ...           ...
37680587  2005           0.0
37680588  2005           0.0
37680589  2005           0.0
37680590  2005           0.0
37680591  2005           0.0

[37680592 rows x 2 columns]


In [ ]:
def dataProcessing(row):
  if pd.isnull(row[1]):
    return ((row[0], "NaN"), 1)
  elif row[1] <= 0:
    return ((row[0], "notDelayed"), 1)
  else:
    return ((row[0], "Delayed"), 1)

location = ["Year", "WeatherDelay"]
rdd = sc.parallelize(flightfile.loc[:, location].values.tolist())
counts = rdd.map(lambda row: dataProcessing(row)) \
      .reduceByKey(lambda a, b: a + b) \
      .collect()

In [ ]:
counts = sorted(counts)
for i in range(len(counts)):
  print(counts[i])

((2000.0, 'NaN'), 5683047)
((2001.0, 'NaN'), 5967780)
((2002.0, 'NaN'), 5271359)
((2003.0, 'Delayed'), 44620)
((2003.0, 'NaN'), 2672742)
((2003.0, 'notDelayed'), 3771178)
((2004.0, 'Delayed'), 115989)
((2004.0, 'notDelayed'), 7013281)
((2005.0, 'Delayed'), 111388)
((2005.0, 'notDelayed'), 7029208)


In [ ]:
import pandas as pd
url = '/content/drive/Shared drives/NCTU-1091-Big Data/Homework 2/2007.csv'
airportfile = pd.read_csv(url, 
            usecols=["ArrDelay", "DepDelay", "Origin", "Dest"],
            dtype={"ArrDelay" : "float", "DepDelay" : "float", "Origin" : "str", "Dest" : "str"}
            )
print(airportfile)

         ArrDelay  DepDelay Origin Dest
0             1.0       7.0    SMF  ONT
1             8.0      13.0    SMF  PDX
2            34.0      36.0    SMF  PDX
3            26.0      30.0    SMF  PDX
4            -3.0       1.0    SMF  PDX
...           ...       ...    ...  ...
7453210      13.0      -7.0    MCO  ATL
7453211       5.0      11.0    ATL  SLC
7453212      15.0      -1.0    LAX  ATL
7453213      36.0      38.0    DFW  ATL
7453214      33.0      24.0    ATL  MCO

[7453215 rows x 4 columns]


In [ ]:
def dataProcessing(row):
  if pd.isnull(row[0]) and pd.isnull(row[1]):
    return [(("Arr", "NaN"), 1), (("Dep", "NaN"), 1)]
  elif pd.isnull(row[0]) and row[1] > 0:
    return [(("Arr", "NaN"), 1), (("Dep", row[3]), 1)]
  elif pd.isnull(row[1]) and row[0] > 0:
    return [(("Arr", row[2]), 1), (("Dep", "NaN"), 1)]
  elif row[0] > 0 and row[1] > 0:
    return [(("Arr", row[2]), 1), (("Dep", row[3]), 1)]
  elif row[0] > 0 and row[1] <= 0:
    return [(("Arr", row[2]), 1), (("Dep", "0"), 1)]
  elif row[0] <= 0 and row[1] > 0:
    return [(("Arr", "0"), 1), (("Dep", row[3]), 1)]
  else:
    return [(("Arr", "0"), 1), (("Dep", "0"), 1)]

location = ["ArrDelay", "DepDelay", "Origin", "Dest"]
rdd = sc.parallelize(airportfile.loc[:, location].values.tolist())
counts = rdd.flatMap(lambda row: dataProcessing(row)) \
      .reduceByKey(lambda a, b: a + b) \
      .collect()

In [ ]:
# class ListNode:
#   def __init__(self, node=None):
#     self.node = node
#     self.nextnode = None
#   def listprint(head):
#     printval = head
#     while printval is not None:
#       print("{}\t{}".format(printval.node[0], printval.node[1]))
#       printval = printval.nextnode
# arrhead = dephead = i = depList = arrList = 0
# while arrhead == 0 or dephead == 0:
#   if counts[i][0][0] == 'Arr' and arrhead == 0:
#     arrList = ListNode([counts[0][1], counts[1]])
#     arrhead = 1
#   elif counts[i][0][0] == 'Dep' and dephead == 0:
#     depList = ListNode([counts[0][1], counts[1]])
#     dephead = 1
#   i+=1
counts = sorted(counts, key=lambda x: x[1], reverse=True)
airport = [[] for i in range(2)]
for i in range(20):
  if counts[i][0][0] == 'Arr':
    airport[0].append([counts[i][0][1], counts[i][1]])
  else:
    airport[1].append([counts[i][0][1], counts[i][1]])
print("Origin\tArrdelay\tDestination\tDepdelay")
for i in range(7):
  print("{}\t{}\t\t{}\t{}".format(airport[0][i][0], airport[0][i][1], airport[1][i][0], airport[1][i][1]))

Origin	Arrdelay	Destination	Depdelay
0	3829420		0	4147653
ATL	209826		NaN	160748
ORD	188410		ATL	156990
NaN	170340		ORD	155461
DFW	148059		DFW	113790
DEN	122557		LAX	103285
LAX	113379		PHX	96496
